# Initial Setup

### $0.1$ Import Necessary Libraries

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### $0.2$ Load the Data

In [21]:
fi_t5_path = "D:/MG/！CUHKSZ/~！大二 下/FIN3080/Assignments/A2/A2_FI_T5(Merge Query).xlsx"
trd_mnth_path = "D:/MG/！CUHKSZ/~！大二 下/FIN3080/Assignments/A2/A2_TRD_Mnth.xlsx"

# Load the financial indicators and trading data
fi_t5 = pd.read_excel(fi_t5_path, header=0, skiprows=[1, 2], dtype={'FI_T5.Stkcd': str})
trd_mnth = pd.read_excel(trd_mnth_path, header=0, skiprows=[1, 2], dtype={'Stkcd': str})

### $0.3$ Prepare the Financial Indicators Data

- **Filtering**: Select only rows where `Typrep == 'A'`, assuming these represent consolidated statements, which are standard for valuation purposes.
- **Column Selection**: Keep `Stkcd` (stock code), `Accper` (quarter-end date), and `F091001A` (Net Assets per Share).
- **Date Conversion**: Convert `Accper` to a datetime format for chronological operations.
- **Sorting**: Sort by `Stkcd` and `Accper` to prepare for merging.
- **Duplicates**: Remove any duplicate entries for the same stock and quarter to ensure data integrity.

In [22]:
# Filter for consolidated statements (Typrep == 'A') and select relevant columns
fi_t5 = fi_t5[fi_t5['FI_T5.Typrep'] == 'A']
fi_t5 = fi_t5[['FI_T5.Stkcd', 'FI_T5.Accper', 'FI_T9.F091001A']]

# Rename columns for simplicity
fi_t5 = fi_t5.rename(columns={
    'FI_T5.Stkcd': 'Stkcd',
    'FI_T5.Accper': 'Accper',
    'FI_T9.F091001A': 'F091001A'
})

# Convert Accper to datetime
fi_t5['Accper'] = pd.to_datetime(fi_t5['Accper'], errors='coerce')
fi_t5 = fi_t5.dropna(subset=['Accper'])

# Sort by stock code and accounting period
fi_t5 = fi_t5.sort_values(by=['Stkcd', 'Accper'])

# Remove duplicates, if any
fi_t5 = fi_t5.drop_duplicates(subset=['Stkcd', 'Accper'])

### $0.4$ Prepare the Trading Data

- **Date Conversion**: The `Trdmnt` column is in 'YYYY-MM' format (e.g., "2010-01"). Convert it to the last day of the month (e.g., "2010-01-31") using `pd.to_datetime` with `MonthEnd(0)`.rows.
- **Sorting**: Sort by `Stkcd` and `Trdmnt` for merging.
- **Duplicates**: Ensure one closing price per stock per month.

In [23]:
# Convert Trdmnt to datetime, representing the last day of the month
trd_mnth['Trdmnt'] = trd_mnth['Trdmnt'].str.extract(r'(\d{4}-\d{2})')[0]
trd_mnth['Trdmnt'] = pd.to_datetime(trd_mnth['Trdmnt'] + '-01') + pd.offsets.MonthEnd(1)

trd_mnth['Year'] = trd_mnth['Trdmnt'].dt.year
trd_mnth['Month'] = trd_mnth['Trdmnt'].dt.month

In [24]:
def get_quarter_end_date(year, month):
    if month <= 3:
        return pd.Timestamp(year, 3, 31)
    elif month <= 6:
        return pd.Timestamp(year, 6, 30)
    elif month <= 9:
        return pd.Timestamp(year, 9, 30)
    else:
        return pd.Timestamp(year, 12, 31)

In [25]:
trd_mnth['Quarter_End_Date'] = trd_mnth.apply(lambda row: get_quarter_end_date(row['Year'], row['Month']), axis=1)

trd_mnth_sorted = trd_mnth.sort_values(['Stkcd', 'Trdmnt'])

In [26]:
# Sort by Trdmnt to satisfy pd.merge_asof requirement
trd_mnth = trd_mnth.sort_values(by='Trdmnt')

# Remove duplicates, if any
trd_mnth = trd_mnth.drop_duplicates(subset=['Stkcd', 'Trdmnt'])

### $0.5$ Merge Data to Match Latest Financial Indicators

Use `pd.merge_asof` to match each `Trdmnt` (month-end date) with the most recent `Accper` (quarter-end date) that is less than or equal to it for each stock.
  - The `direction='backward'` ensures we take the latest available financial data before or at the trading month.
    - For example, for November 2019 ("2019-11-30"), it uses the Net Assets per Share from Q3 2019 ("2019-09-30").

In [ ]:
# Merge trading data with financial data using as-of merge (backward)
merged = pd.merge_asof(
    trd_mnth,
    fi_t5,
    left_on='Trdmnt',
    right_on='Accper',
    by='Stkcd',
    direction='backward'
)

print(merged.head())

ValueError: right keys must be sorted

### $0.6$ Calculate P/B Ratios

The P/B ratio is computed by dividing `Mclsprc` by `F091001A`. Ignore the existing `F100401A` (P/B Ratio) column as per the task requirement to calculate it manually.

In [ ]:
# Calculate P/B ratio as Monthly Closing Price / Net Assets per Share
merged['PB_ratio'] = merged['Mclsprc'] / merged['F091001A']

### $0.7$ Filter by Date Range

Restrict the data to the specified period. Since `Trdmnt` is the last day of each month, "2010-01-31" to "2024-12-31" covers January 2010 to December 2024.